In [16]:
%cd ..

d:\Documents\ProjectsCompany\Kata.ai\Research\2021\NewLM


In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from newlm.lm.elmo.modeling_elmo.elmo_model import ELMOGPTHeadModel
from newlm.lm.elmo.modeling_elmo.elmo_config import ELMOConfig

In [19]:
from transformers import GPT2Config

In [20]:
from transformers import GPT2Tokenizer

In [21]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [40]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [61]:
tokenizer.pad_token_id

50257

In [62]:
config = GPT2Config(vocab_size=55000, pad_token_id=tokenizer.pad_token_id)

In [63]:
config.pad_token_id

50257

In [64]:
elmo_config = ELMOConfig(pad_token_id=tokenizer.pad_token_id,lr2_gpt_config=config, r2l_gpt_config=config)


In [65]:
elmo_model = ELMOGPTHeadModel(elmo_config)

In [66]:
batch = tokenizer.batch_encode_plus(['my name is haryo', 'i am a student'], return_tensors='pt', padding=True)

In [74]:
elmo_model(**batch)

> d:\documents\projectscompany\kata.ai\research\2021\newlm\newlm\newlm\lm\elmo\modeling_elmo\elmo_model.py(119)forward()
    117         breakpoint()
    118 
--> 119         return ElmoGPTCausalLMOutput(
    120             loss=loss,
    121             l2r_last_hidden_state=l2r_outs.hidden_states,



BdbQuit: 

In [73]:
elmo_config.r2l_gpt_config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50257,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.10.0",
  "use_cache": true,
  "vocab_size": 55000
}

In [ ]:
elmo_model(**batch)

AttributeError: 'list' object has no attribute 'size'